# Uses kernel at  https://github.com/Carglglz/jupyter_upydevice_kernel

In [ ]:
%websocketconnect 192.168.1.130:8266 --password horsetoe

### If webrepl not working, can try
`%serialconnect /dev/tty.SLAB_USBtoUART 115200`

In [ ]:
import os
print(os.listdir())

See the current boot.py on the esp8266 device

In [ ]:
with open('boot.py') as f:
    print(f.read())

In [ ]:
with open('credentials.json', 'r') as f:
    credentials = ujson.load(f)
f.close()

In [ ]:
credentials

Rather than block on waiting for wifi, following https://stackoverflow.com/questions/567622/is-there-a-pythonic-way-to-try-something-up-to-a-maximum-number-of-times to try to connect 12 times

## Write a boot script that you want to write to flash

In [ ]:
script = """import network
import webrepl
import utime
import ntptime
import machine
import ujson
import gc

with open('credentials.json', 'r') as f:
    credentials = ujson.load(f)
f.close()

def do_connect():
    print("Connecting to ", credentials['ssid'])
    wifi_interface = network.WLAN(network.STA_IF)
    if not wifi_interface.isconnected():
        wifi_interface.active(False)
        print('First connection attempt...')
        wifi_interface.active(True)
        wifi_interface.ifconfig(('192.168.1.130','255.255.255.0','192.168.1.1','8.8.8.8'))
        wifi_interface.connect(credentials['ssid'], credentials['password'])
    utime.sleep(0.5)
    connect_attempts = 0
    frequency = 5000
    led = machine.PWM(machine.Pin(2), frequency)
    while connect_attempts < 12:
        led.duty(0)
        print('check wifi status')
        if wifi_interface.isconnected():
            webrepl.start()
            print('network config:', wifi_interface.ifconfig())
            for i in range(5):
                led.duty(0)
                utime.sleep(0.3)
                led.duty(1023)
                utime.sleep(0.3)
            break
        else:
            connect_attempts += 1
            wifi_interface.active(False)
            print('reattempting connection')
            wifi_interface.active(True)
            wifi_interface.connect(credentials['ssid'], credentials['password'])
            for duty_cycle in range(0, 1024):
                led.duty(duty_cycle)
                utime.sleep(0.003)
            utime.sleep(0.5)

do_connect()
utime.sleep(2)
ntptime.settime()
gc.collect()
"""

### First write to a holder location

In [ ]:
with open('boot.txt', 'w') as f:
    f.write(script)

### Double check the write

In [ ]:
with open('boot.txt') as f:
    print(f.read())

In [ ]:
os.remove('boot.py')

## Move temp file to `boot.py`

In [ ]:
os.rename('boot.txt', 'boot.py')

In [ ]:
%disconnect

In [ ]:
%rebootdevice

## Testing

### Try async webrepl

In [ ]:
import uasyncio, uaiowebrepl, uaiorepl
loop = uasyncio.get_event_loop()
loop.create_task(uaiowebrepl.start(ip="192.168.1.130", port=8266, password="horsetoe"))
loop.create_task(uaiorepl.start()) # uasyncio repl will process webrepl input
loop.run_forever()

In [ ]:
2+2

Test networking

In [ ]:
wifi_interface = network.WLAN(network.STA_IF)

In [ ]:
wifi_interface.isconnected()

In [ ]:
wifi_interface.ifconfig()

In [ ]:
ap_if = network.WLAN(network.AP_IF)

In [ ]:
ap_if.active(False)

In [ ]:
wifi_interface.active(True)

In [ ]:
wifi_interface.ifconfig(('192.168.1.130','255.255.255.0','192.168.1.1','8.8.8.8'))

In [ ]:
wifi_interface.connect("Nevertheless", "obama4ever")

In [ ]:
wifi_interface.isconnected()